## Titanic Disaster Dataset Feature Engineering

This notebook is an attempt towards kick-starting my way around the Titanic dataset to improve my model scores based on feature engineering techniques.
This is not going to be an extensive EDA notebook for viewers, as there are already tons of notebooks available with great amount of EDA to understand the intricacies of the dataset from all aspects.
* We will work with missing value imputations
* We will also try to combine some features with each other to come up with new ones
* We will try to improve our scores by building up on our feature engineering process

In [1]:
# Import the required packages here

import numpy as np
import pandas as pd

In [2]:
# Import the datasets here
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [3]:
print(train.shape)
print(test.shape)

(891, 12)
(418, 11)


In [4]:
# Append the train and test dataframes for data cleaning

train['test_flag'] = 0
test['test_flag'] = 1
df_combined = pd.concat([train, test], axis=0, copy=True)

### High level EDA

In [5]:
# Check the % missing values in all the columns of the train set
print(df_combined.isnull().sum()*100/df_combined.shape[0])

# Ignore the missing values for the output class 'Survived' as they are from the test set

PassengerId     0.000000
Survived       31.932773
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            20.091673
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.076394
Cabin          77.463713
Embarked        0.152788
test_flag       0.000000
dtype: float64


In [6]:
# Subsetting for the list of columns which has less to no missing values

df_subset = df_combined[['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Name', 'Embarked', 'test_flag']]

### Data Cleaning and missing value imputation for the columns

In [7]:
# Cleaning and level modifications for the categorical features

for dataset in df_subset:
    df_subset['Title'] = df_subset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(df_subset['Title'], df_subset['Sex'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Sex,female,male
Title,,
Capt,0,1
Col,0,4
Countess,1,0
Don,0,1
Dona,1,0
Dr,1,7
Jonkheer,0,1
Lady,1,0
Major,0,2


In [8]:
for dataset in df_subset:
    df_subset['Title'] = df_subset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
     'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    df_subset['Title'] = df_subset['Title'].replace('Mlle', 'Miss')
    df_subset['Title'] = df_subset['Title'].replace('Ms', 'Miss')
    df_subset['Title'] = df_subset['Title'].replace('Mme', 'Mrs')
    
df_subset[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

,Title,Survived
0,Master,0.575000
1,Miss,0.702703
2,Mr,0.156673
3,Mrs,0.793651
4,Rare,0.347826


In [9]:
df_subset = df_subset.drop(['Name'], axis = 1)

In [10]:
df_subset['Cabin'] = df_subset['Cabin'].replace(np.nan, 'U')
df_subset['Cabin_Class'] = df_subset['Cabin'].astype(str).str[0]

In [11]:
df_subset = df_subset.drop(['Ticket', 'Cabin'], axis = 1)

In [12]:
median_age = df_subset["Age"].median()
df_subset["Age"].fillna(median_age, inplace=True)

In [13]:
# df_subset.Age[df_subset.Age <= 12] = 0
# df_subset.Age[(df_subset.Age > 12) & (df_subset.Age <= 30)] = 1
# df_subset.Age[(df_subset.Age > 30) & (df_subset.Age <= 50)] = 2
# df_subset.Age[(df_subset.Age > 50) & (df_subset.Age <= 65)] = 3
# df_subset.Age[df_subset.Age > 65] = 4

In [14]:
df_subset['Age'] = df_subset['Age'].astype(int)

In [15]:
df_subset.Age[df_subset.Age < 18] = 1
df_subset.Age[df_subset.Age >= 18] = 0

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
for dataset in df_subset:
    df_subset['FamilySize'] = df_subset['SibSp'] + df_subset['Parch'] + 1

In [17]:
df_subset = df_subset.drop(['SibSp', 'Parch'], axis = 1)

In [18]:
df_subset.FamilySize[df_subset.FamilySize == 1] = 0
df_subset.FamilySize[(df_subset.FamilySize > 1) & (df_subset.FamilySize <= 4)] = 1
df_subset.FamilySize[(df_subset.FamilySize > 4)] = 2

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [19]:
df_subset['Fare'].describe()

count    1308.000000
mean       33.295479
std        51.758668
min         0.000000
25%         7.895800
50%        14.454200
75%        31.275000
max       512.329200
Name: Fare, dtype: float64

In [20]:
df_subset['Fare'].fillna(df_subset['Fare'].dropna().median(), inplace=True)

In [21]:
df_subset.Fare[df_subset.Fare <= 7.91] = 0
df_subset.Fare[(df_subset.Fare > 7.91) & (df_subset.Fare <= 14.454)] = 1
df_subset.Fare[(df_subset.Fare > 14.454) & (df_subset.Fare <= 31)] = 2
df_subset.Fare[df_subset.Fare > 31] = 3
df_subset['Fare'] = df_subset['Fare'].astype(int)

df_subset.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,test_flag,Title,Cabin_Class,FamilySize
0,1,0.0,3,male,0,0,S,0,Mr,U,1
1,2,1.0,1,female,0,3,C,0,Mrs,C,1
2,3,1.0,3,female,0,1,S,0,Miss,U,0
3,4,1.0,1,female,0,3,S,0,Mrs,C,1
4,5,0.0,3,male,0,1,S,0,Mr,U,0


In [22]:
mode_embarked = df_subset.Embarked.dropna().mode()[0]
df_subset["Embarked"].fillna(mode_embarked, inplace=True)

In [23]:
df_subset.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,test_flag,Title,Cabin_Class,FamilySize
0,1,0.0,3,male,0,0,S,0,Mr,U,1
1,2,1.0,1,female,0,3,C,0,Mrs,C,1
2,3,1.0,3,female,0,1,S,0,Miss,U,0
3,4,1.0,1,female,0,3,S,0,Mrs,C,1
4,5,0.0,3,male,0,1,S,0,Mr,U,0


In [24]:
cols_new = ['Embarked', 'Title', 'Sex', 'Cabin_Class']

for col in cols_new:
    df_subset[col] = pd.factorize(df_subset[col])[0] + 1
    
df_subset.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,test_flag,Title,Cabin_Class,FamilySize
0,1,0.0,3,1,0,0,1,0,1,1,1
1,2,1.0,1,2,0,3,2,0,2,2,1
2,3,1.0,3,2,0,1,1,0,3,1,0
3,4,1.0,1,2,0,3,1,0,2,2,1
4,5,0.0,3,1,0,1,1,0,1,1,0


In [25]:
train_set = df_subset[df_subset['test_flag']==0]
test_set = df_subset[df_subset['test_flag']==1]

In [26]:
train_set['Survived'] = train_set['Survived'].astype(int)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
print(train_set.shape)
print(test_set.shape)

(891, 11)
(418, 11)


In [28]:
test_set = test_set.drop(['Survived', 'test_flag'], axis = 1)
train_set = train_set.drop(['test_flag'], axis = 1)

In [29]:
train_set.head(5)

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Title,Cabin_Class,FamilySize
0,1,0,3,1,0,0,1,1,1,1
1,2,1,1,2,0,3,2,2,2,1
2,3,1,3,2,0,1,1,3,1,0
3,4,1,1,2,0,3,1,2,2,1
4,5,0,3,1,0,1,1,1,1,0


In [30]:
from sklearn.model_selection import train_test_split

X = train_set[['Age', 'Sex', 'Embarked', 'Title', 'Pclass', 'FamilySize']]
y = train_set[['Survived']]

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(596, 6) (596, 1)
(295, 6) (295, 1)


In [32]:
X_test.head()

,Age,Sex,Embarked,Title,Pclass,FamilySize
709,0,1,2,4,3,1
439,0,1,1,1,2,0
840,0,1,1,1,3,0
720,1,2,1,3,2,1
39,1,2,2,3,3,1


In [33]:
# Logistic Regression Model

from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logistic_model = logreg.fit(X_train, y_train)
predictions = logistic_model.predict(X_test)

acc_logistic_regressions = round(logistic_model.score(X_train, y_train) * 100, 2)
acc_logistic_regressions

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


80.2

In [34]:
# Decision Tree model
from sklearn.tree import DecisionTreeClassifier

dec_tree = DecisionTreeClassifier()
decision_tree = dec_tree.fit(X_train, y_train)
prediction_tree = decision_tree.predict(X_test)

acc_decision_tree = round(decision_tree.score(X_train, y_train) * 100, 2)
acc_decision_tree

85.57

In [35]:
# Random Forest Model

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
random_forest = rf.fit(X_train, y_train)
prediction_forest = random_forest.predict(X_test)

acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)
acc_random_forest

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


85.57

In [36]:
# Naive Bayes

from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gaussian = gnb.fit(X_train, y_train)
prediction_naive = gaussian.predict(X_test)

acc_gaussian = round(gaussian.score(X_train, y_train) * 100, 2)
acc_gaussian

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


77.68

In [37]:
# Perceptron Model

from sklearn.linear_model import Perceptron

pcp = Perceptron()
perceptron = pcp.fit(X_train, y_train)
prediction_perceptron = perceptron.predict(X_test)

acc_perceptron = round(perceptron.score(X_train, y_train) * 100, 2)
acc_perceptron

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


79.03

### Use the section below to submit on the test set

In [38]:
test_feature = test_set[['Age', 'Sex', 'Embarked', 'Title', 'Pclass', 'FamilySize']]
test_id = test_set['PassengerId']
test_feature.head()

,Age,Sex,Embarked,Title,Pclass,FamilySize
0,0,1,3,1,3,0
1,0,2,1,2,3,1
2,0,1,3,1,2,0
3,0,1,1,1,3,0
4,0,2,1,2,3,1


In [39]:
test_predictions = random_forest.predict(test_feature)

In [40]:
output = pd.DataFrame({'PassengerId' : test_id, 'Survived': test_predictions})
output.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [41]:
# Use this data type conversion for the output feature, if the score comes as 0 in submission

# model.predict(test_data).astype(int)

In [42]:
output.to_csv('/kaggle/working/submission.csv', index=False)